In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from datetime import date, timedelta
from pandas.tseries.offsets import BDay
from scipy import stats
from tqdm import tqdm
%matplotlib inline

/Users/mattevanoff/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/mattevanoff/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
dat = pd.read_csv('data/train_val.csv', names=['account_id', 'series_id', 'posted_at', 'amount_cents'], 
                  parse_dates=['posted_at'])
print('rows: {}, columns: {}'.format(*dat.shape))
print('unique series_id: {}'.format(dat.series_id.nunique()))
# round to day
dat = dat.assign(posted_at=dat.posted_at.dt.floor('D'))
dat.head()

rows: 374508, columns: 4
unique series_id: 51534


,account_id,series_id,posted_at,amount_cents
0,54ad727ddc76040900d55051,54ad727ddc76040900d55051|82267c03919ef18dabc33...,2017-09-27,2754
1,54b80cb0cb084b0c00294f2a,54b80cb0cb084b0c00294f2a|6ce908fc9895fdfc035e4...,2018-06-08,538
2,54bde3073deefe0c00a01401,54bde3073deefe0c00a01401|46e37148a68739387c48b...,2017-11-28,1614
3,54c6ea8213078c0c001cec71,54c6ea8213078c0c001cec71|044a408e374c8a102b709...,2018-04-02,22105
4,54e643cd95962f0c004cde71,54e643cd95962f0c004cde71|ee74f9800708efd6b39c3...,2018-05-29,7803


### Helper functions

In [3]:
# Create pivot table for dates 
def create_pivot(column, fillna=False):
    time_dat = dat.copy()
    time_dat.set_index(['posted_at'], inplace=True)
    idx = pd.date_range(dat.posted_at.min(), dat.posted_at.max())
    time_dat.index = pd.DatetimeIndex(time_dat.index)
    time_dat.reset_index(inplace=True)
    time_dat = time_dat.pivot_table(values='amount_cents', index=column, columns='posted_at')
    if fillna: time_dat = time_dat.fillna(0)
    return time_dat

def create_pivot_BS(column, fillna=False, use_bs_days_only=False):
    time_dat = dat.copy()
    time_dat.set_index(['posted_at'], inplace=True)
    idx = pd.date_range(dat.posted_at.min(), dat.posted_at.max(), freq=BDay())
    time_dat.index = pd.DatetimeIndex(time_dat.index)
    time_dat.reset_index(inplace=True)
    time_dat = time_dat.pivot_table(values='amount_cents', index=column, columns='posted_at')
    if fillna: time_dat = time_dat.fillna(0)
    if use_bs_days_only: time_dat = time_dat[idx]
    return time_dat

from pandas.api.types import is_string_dtype, is_numeric_dtype
from sklearn.preprocessing import StandardScaler
import warnings
from sklearn.exceptions import DataConversionWarning
from sklearn_pandas import DataFrameMapper
warnings.filterwarnings('ignore')
import re

def transform_date(df, field_name, drop=True):
    field = df[field_name]
    if not np.issubdtype(field, np.datetime64):
        df[field_name] = field = pd.to_datetime(field, infer_datetime_format=True)
    target_pre = re.sub('[Dd]ate$', '', field_name)
#     for i in ('Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear'):
    for i in ('Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start'):
        df[target_pre + i] = getattr(field.dt, i.lower())
    df[target_pre + 'Elapsed'] = field.astype(np.int64) // 10**9
    if drop:
        df.drop(field_name, axis=1, inplace=True)


# from a time (dt) go back (minus) day to get data from days a certain number of days (periods)
def get_timespan(df, dt, minus, periods, freq='D'):
    return df[pd.date_range(dt - timedelta(days=minus), periods=periods, freq=freq)]
    
def plot_individual(id_, df, date_range=None):
    fig, ax = plt.subplots()
    sample = df.loc[id_]
    if date_range:
        # example '2017-09-16', '2017-09-20'
        sample = sample.loc[pd.date_range(date_range[0], date_range[1])]
    sample.plot(kind='bar', figsize=(20, 5), ax=ax)
    for label, value in zip(ax.xaxis.get_ticklabels(), sample):
        if value == 0 or np.isnan(value):
            label.set_visible(False)
    plt.title('Transactions for {}'.format(id_))
    plt.ylabel('Amount in Cents')
    plt.tight_layout()

In [4]:
series = create_pivot_BS('series_id')
series.head()

posted_at,2017-09-14 00:00:00,2017-09-15 00:00:00,2017-09-16 00:00:00,2017-09-17 00:00:00,2017-09-18 00:00:00,2017-09-19 00:00:00,2017-09-20 00:00:00,2017-09-21 00:00:00,2017-09-22 00:00:00,2017-09-23 00:00:00,...,2018-06-11 00:00:00,2018-06-12 00:00:00,2018-06-13 00:00:00,2018-06-14 00:00:00,2018-06-15 00:00:00,2018-06-16 00:00:00,2018-06-17 00:00:00,2018-06-18 00:00:00,2018-06-19 00:00:00,2018-06-20 00:00:00
series_id,,,,,,,,,,,,,,,,,,,,,
53a85331e431960200466a39|140fae5cfea3485ab5c6d7f7266677ba|0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53a85378e431960200469d42|76f7222aa18d6b39a7ebdd5168b1ffd4|0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53a85378e431960200469d42|ff242a09ec5b7f16a4941334c39783ea|0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,200.0,200.0,200.0,NaN,200.0,NaN,NaN,200.0,100.0,NaN
53a85392e43196020046b36b|76d3583b46575283f21d8cd598659d4c|0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53a853aee43196020046c978|9bd3a9269b8f1d37371e5b0205bb67df|0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Create time periods / days between transactions

In [5]:
%%time
df = pd.DataFrame(index=series.index)
# series_id, then days till next purchase
all_data_sequences = []
for ser in series.index:
    day_sequence = []
    sample = series.loc[ser]
    non_null = pd.DataFrame(sample[sample.notnull()]).T
    for i in range(len(non_null.columns)-1):
        cur_date = non_null.columns[i]
        next_date = non_null.columns[i+1]
        diff = pd.Timedelta(next_date - cur_date).days
        day_sequence.append(diff)
    all_data_sequences.append(day_sequence)
df['day_sequence'] = all_data_sequences

CPU times: user 57.9 s, sys: 697 ms, total: 58.6 s
Wall time: 60 s


### Add actual dates to df

In [6]:
%%time
# series_id, then days till next purchase
all_time_sequences = []
for ser in series.index:
    day_sequence = []
    sample = series.loc[ser]
    non_null = pd.DataFrame(sample[sample.notnull()]).T
    for i in range(len(non_null.columns)):
        cur_date = non_null.columns[i]
#         next_date = non_null.columns[i+1]
#         diff = pd.Timedelta(next_date - cur_date).days
        day_sequence.append(cur_date)
    all_time_sequences.append(day_sequence)
df['time_sequence'] = all_time_sequences

CPU times: user 48.9 s, sys: 668 ms, total: 49.5 s
Wall time: 50.8 s


### create final prediction dataframe and simple features
- This turned out to be the highest score on the validation set, more ideas in the EDA and Ideas which should be explored more if I had more time 

In [7]:
final_df = df.copy()
final_df['mean'] = final_df.day_sequence.map(lambda x: np.floor(np.mean(x)))
final_df['mode'] = final_df.day_sequence.map(lambda x: stats.mode(x)[0][0] if len(x) > 0 else stats.mode(x)[0])
final_df['X_time_max'] = final_df.time_sequence.map(lambda x: x[-1])

In [8]:
def map_to_date(row):
    d = date(row['X_time_max'].year, row['X_time_max'].month, row['X_time_max'].day) 
    if isinstance(row['mode'], int):
        col = timedelta(row['mode'])
        return pd.Timestamp(d + col)
    return pd.Timestamp(d + timedelta(28))
 
final_df['pred_date'] = final_df.apply(map_to_date, axis=1)

### Predict Amount Cents

In [9]:
def exponential_smoothing(panda_series, alpha_value):
    ouput=sum([alpha_value * (1 - alpha_value) ** i * x for i, x in enumerate(panda_series['amount_cents'])])
    return ouput

In [11]:
%%time 
all_amount_preds = []
for sample_series_id in tqdm(dat.series_id.unique()):
    panda_series = dat[dat.series_id == sample_series_id][['posted_at', 'amount_cents']].sort_values('posted_at')
    smoothing_number = exponential_smoothing(panda_series, 0.7) 
    all_amount_preds.append(smoothing_number)

### Create final df for predictions 

In [ ]:
submission_df = final_df.reset_index()[['series_id', 'pred_date']]
submission_df.columns = ['series_id', 'posted_at']
submission_df['amount_cents'] = all_amount_preds
submission_df['amount_cents'] = np.round(all_amount_preds)
submission_df.to_csv('predictions.csv', index=False)